In [ ]:
import gc
gc.collect()

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [1]:
import numpy as np
import skimage.measure
# import pydicom
import vtk
from vtk.util import numpy_support
import os
import matplotlib.pyplot as plt
import cv2

In [2]:
plt.gray()
os.chdir('C:/Users/span/Documents/3DSlicerTutorial/CNN.test/')
number1 = "23"

<Figure size 432x288 with 0 Axes>

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
)
        
val_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
path1 = "data/train_image/23_train_image"
path2 = "data/train_mask/23_train_mask"
path3 = "data/val_image/23_test_image"
path4 = "data/val_mask/23_test_mask"
batch_size1 = 16

train_image_generator = train_datagen.flow_from_directory(
    path1,
    color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)

train_mask_generator = train_datagen.flow_from_directory(
    path2,
    color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)

val_image_generator = val_datagen.flow_from_directory(
    path3,
    color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)


val_mask_generator = val_datagen.flow_from_directory(
    path4,
    color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)

train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

Found 244 images belonging to 1 classes.
Found 488 images belonging to 2 classes.
Found 176 images belonging to 1 classes.
Found 352 images belonging to 2 classes.


In [6]:
from keras import backend as K
import keras
# from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Input
import tensorflow as tf

def get_model(optimizer, loss_metric, metrics, lr=1e-3):
    inputs = keras.Input((sample_width, sample_height, 1))
    conv1 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(inputs)
    conv1 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(conv1)
    conv1 = tf.reshape(conv1, tf.shape(conv1))
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    drop1 = keras.layers.Dropout(0.5)(pool1)

    conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(drop1)
    conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(conv2)
    conv2 = tf.reshape(conv2, tf.shape(conv2))
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    drop2 = keras.layers.Dropout(0.5)(pool2)

    conv3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(drop2)
    conv3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(conv3)
    conv3 = tf.reshape(conv3, tf.shape(conv3))
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    drop3 = keras.layers.Dropout(0.3)(pool3)

    conv4 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(drop3)
    conv4 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(conv4)
    conv4 = tf.reshape(conv4, tf.shape(conv4))
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
    drop4 = keras.layers.Dropout(0.3)(pool4)

    conv5 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='SAME')(drop4)
    conv5 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='SAME')(conv5)
    conv5 = tf.reshape(conv5, tf.shape(conv5))
    
    conv5u = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv5)
    conv5u = tf.reshape(conv5u, tf.shape(conv5u))
    up6 = keras.layers.concatenate([conv5u, conv4], axis=3)
    conv6 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(up6)
    conv6 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(conv6)
    conv6 = tf.reshape(conv6, tf.shape(conv6))
    
    conv6u = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv6)
    conv6u = tf.reshape(conv6u, tf.shape(conv6u))
    up7 = keras.layers.concatenate([conv6u, conv3], axis=3)
    conv7 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(up7)
    conv7 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(conv7)
    conv7 = tf.reshape(conv7, tf.shape(conv7))
    
    conv7u = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv7)
    conv7u = tf.reshape(conv7u, tf.shape(conv7u))
    up8 = keras.layers.concatenate([conv7u, conv2], axis=3)
    conv8 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(up8)
    conv8 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(conv8)
    conv8 = tf.reshape(conv8, tf.shape(conv8))
    
    conv8u = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv8)
    conv8u = tf.reshape(conv8u, tf.shape(conv8u))
    up9 = keras.layers.concatenate([conv8u, conv1], axis=3)
    conv9 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(up9)
    conv9 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(conv9)
    conv9 = tf.reshape(conv9, tf.shape(conv9))

    conv10 = keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    conv10 = tf.reshape(conv10, tf.shape(conv10))

    model = keras.Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=optimizer(lr=lr), loss=loss_metric, metrics=metrics)
    return model

In [7]:
smooth = 1.
# Dice Coefficient to work with Tensorflow
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

# Dice Coefficient to work outside Tensorflow
def dice_coef_2(y_true, y_pred):
    side = len(y_true[0])
    y_true_f = y_true.reshape(side*side)
    y_pred_f = y_pred.reshape(side*side)
    intersection = sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (sum(y_true_f) + sum(y_pred_f) + smooth)

In [8]:
sample_width = 256
sample_height = 256

model = get_model(optimizer=tf.keras.optimizers.Adam, loss_metric=dice_coef_loss, metrics=['AUC', dice_coef], lr=1e-6)

In [ ]:
# model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# historyTF = model.fit(
#     train_generator, 
#     batch_size=4, 
#     epochs=10, 
#     validation_data = 
#     (val_image_generator,
#      val_mask_generator)
# )

In [ ]:
# fit the keras model on the dataset
# historyKe = model.fit(train_image_generator, train_mask_generator, epochs=150, batch_size=10)

# train_generator = zip(train_image_generator, train_mask_generator)
# val_generator = zip(val_image_generator, val_mask_generator)

# from keras import backend as K

# historyTF = model.fit(train_generator, epochs=10, validation_data=val_generator)


In [ ]:
NO_OF_EPOCHS = 50
NO_OF_TRAINING_IMAGES = len(os.listdir('C:/Users/span/Documents/3DSlicerTutorial/CNN.test/data/train_image/23_train_image/train/'))
NO_OF_VAL_IMAGES = len(os.listdir('C:/Users/span/Documents/3DSlicerTutorial/CNN.test/data/val_image/23_test_image/val/'))
BATCH_SIZE = batch_size1

results = model.fit(train_generator, 
                    epochs=NO_OF_EPOCHS, 
                    steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE), 
                    validation_data=val_generator, 
                    validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE)
                   )

Epoch 1/50


In [ ]:
model.save('Model_ep100.h5')

In [ ]:
# !mkdir saved_model
model.save('saved_model/my_model_ep100') 

In [ ]:
print(results.history.keys())

In [ ]:
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(results.history['auc'])
plt.plot(results.history['val_auc'])
plt.title('model auc')
plt.ylabel('auc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
from PIL import Image

# img0 = Image.fromarray(val_image_generator[3].astype(np.uint8), mode = "L")

im = Image.new('L', (256, 256))
im.putdata(val_image_generator[5])
plt.imshow(im)

In [ ]:
dataset_as_tuple = next(val_image_generator)


samples_to_predict = np.array(dataset_as_tuple[0])


In [ ]:
predictions = model.predict(dataset_as_tuple[0])
plt.imshow(np.array(dataset_as_tuple[0])[0,:,:].astype(np.uint8))

In [ ]:
a1 = np.array(dataset_as_tuple[0])[0,:,:]
plt.imshow(a1)

In [ ]:
print(a1.max(),a1.min())